<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=8efa1c4b9a23ebdddc409db8249451df594493177792e7a7ed5d55bd6c2d05f9
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-28 14:24:58
-------------------
qualified stocks: 89
with latest results: 28
still star stocks: 17
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  2.00 K
Current:  1.45 C
-------------------
Today PnL: -60.45 K (-0.42%)
Current PnL: -19.63 L (-12.86%)
CY Booked + Current PnL: -7.53 L (-4.93%)
-------------------
Total profit:  2.40 L
Total loss:  -22.03 L
-------------------
Total Booked + Current PnL: 19.45 L (15.48%)
Total Booked PnL: 39.08 L (31.11%)
Curr Year Booked PnL: 12.10 L (8.37%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 87.53 L (60.55%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.36%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,120.92,78.0,M-SC,1.64,95139.0,-5638.0,5727.0,12.26,-5.59,6.02,0.09,245.0,-0.98,0.66,24.99,OX40N,NTT,DURABLES
59,RELIANCE,1533.0,-7.66,72.0,X-LC,3.83,231426.0,21060.0,7730.0,-0.04,10.01,3.34,13.68,37.0,2.72,1.60,28.19,XY25,NTT,REFINERIES
2,ABBOTINDIA,35195.0,-9.24,44.0,X-MC,2.98,89730.0,-948.0,15855.0,0.47,-1.05,17.67,16.44,101.0,-0.06,0.62,18.92,X40,ATH,PHARMA
86,VOLTAS,1530.0,1.86,55.0,X-MC,3.57,213570.0,21828.0,15932.0,-0.89,11.38,7.46,19.69,99.0,1.37,1.48,18.96,XY25,NTT,AC
80,UJJIVANSFB,60.0,129.87,73.0,H-SC,17.48,145368.0,2889.0,16630.0,-0.30,2.03,11.44,13.70,163.0,0.17,1.01,71.57,OX40N,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,120.92,78.0,M-SC,1.64,95139.0,-5638.0,5727.0,12.26,-5.59,6.02,0.09,245.0,-0.98,0.66,24.99,OX40N,NTT,DURABLES
39,INDUSINDBK,1800.0,271.15,68.0,L-MC,9.21,45554.0,-31652.0,57047.0,3.78,-41.00,125.23,32.89,258.0,-0.55,0.32,25.45,XR,NTT,BANKS
84,VALIANTORG,838.0,-291.52,46.0,H-SC,8.66,106556.0,-57049.0,178364.0,2.86,-34.87,167.39,74.15,139.0,-0.32,0.74,36.59,XR,NTT,CHEMICALS
83,VAIBHAVGBL,521.0,55.92,60.0,H-SC,4.41,134976.0,-47799.0,158867.0,2.47,-26.15,117.70,60.77,125.0,-0.30,0.93,20.89,XR,NTT,JEWELLERY
54,QUESS,986.0,-44.90,52.0,X-SC,38.35,54509.0,-10497.0,161423.0,2.32,-16.15,296.14,232.18,198.0,-0.07,0.38,6.32,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,2096.00,-37.87,35.0,X-SC,4.48,92877.0,-35793.0,81091.0,-4.05,-27.82,87.31,35.20,219.0,-0.44,0.64,6.60,X40,NTT,FOOTWEAR
32,HINDZINC,730.22,25.32,44.0,M-LC,11.15,204015.0,-1061.0,113636.0,-2.76,-0.52,55.70,54.89,52.0,-0.01,1.41,23.45,X5K,ATH,METALS
15,BSOFT,836.99,-14.41,59.0,H-SC,13.96,96262.0,-44391.0,117170.0,-2.38,-31.56,121.72,51.74,131.0,-0.38,0.67,11.37,XR,ATH,IT
8,ATULAUTO,844.00,3548.65,45.0,M-SC,3.41,163744.0,-33388.0,123217.0,-2.29,-16.94,75.25,45.57,236.0,-0.27,1.13,16.09,XY24,NTT,AUTO
5,ANGELONE,3033.00,16.84,65.0,X-SC,5.34,206640.0,15634.0,42072.0,-2.21,8.19,20.36,30.21,157.0,0.37,1.43,30.78,X40N,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,43.54,52.0,M-MC,9.74,226977.0,2015.0,165648.0,1.47,0.90,72.98,74.53,192.0,0.01,1.57,33.44,XY24,BTT,STEEL
36,IEX,219.00,-30.16,63.0,H-SC,16.42,199832.0,-122.0,97358.0,0.08,-0.06,48.72,48.63,115.0,-0.00,1.38,11.48,XR,NTT,MISC
57,RECLTD,446.00,42.86,47.0,M-LC,5.50,202455.0,-715.0,42839.0,-1.50,-0.35,21.16,20.74,55.0,-0.02,1.40,6.49,XY25,NTT,FINANCE
88,WIPRO,420.00,-14.70,44.0,M-LC,5.79,149767.0,-1178.0,110633.0,-0.96,-0.78,73.87,72.51,53.0,-0.01,1.04,5.20,XR,NTT,IT
32,HINDZINC,730.22,25.32,44.0,M-LC,11.15,204015.0,-1061.0,113636.0,-2.76,-0.52,55.70,54.89,52.0,-0.01,1.41,23.45,X5K,ATH,METALS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,UJJIVANSFB,60.0,129.87,73.0,H-SC,17.48,145368.0,2889.0,16630.0,-0.30,2.03,11.44,13.70,163.0,0.17,1.01,71.57,OX40N,NTT,BANKS
79,TTKPRESTIG,770.0,120.92,78.0,M-SC,1.64,95139.0,-5638.0,5727.0,12.26,-5.59,6.02,0.09,245.0,-0.98,0.66,24.99,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.61,48.0,H-SC,2.57,223137.0,-46530.0,82873.0,-2.07,-17.25,37.14,13.47,138.0,-0.56,1.54,13.46,XY24,NTT,PAINTS
74,TATAELXSI,9161.0,-22.20,54.0,H-MC,7.05,105165.0,-26670.0,68894.0,-0.90,-20.23,65.51,32.03,98.0,-0.39,0.73,17.79,OX40N,NTT,IT
68,SIS,528.0,2058.03,50.0,H-SC,3.14,87082.0,-23950.0,47555.0,-0.15,-21.57,54.61,21.26,156.0,-0.50,0.60,17.66,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.0,-15.53,75.0,H-MC,15.22,216520.0,36712.0,75695.0,0.77,20.42,34.96,62.52,88.0,0.48,1.5,61.46,XR,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,UJJIVANSFB,60.00,129.87,73.0,H-SC,17.48,145368.0,2889.0,16630.0,-0.30,2.03,11.44,13.70,163.0,0.17,1.01,71.57,OX40N,NTT,BANKS
62,SAIL,228.00,43.54,52.0,M-MC,9.74,226977.0,2015.0,165648.0,1.47,0.90,72.98,74.53,192.0,0.01,1.57,33.44,XY24,BTT,STEEL
47,KPIGREEN,731.05,9.40,69.0,H-SC,5.28,130498.0,5201.0,53726.0,-0.83,4.15,41.17,47.03,141.0,0.10,0.90,63.40,MH,ATH,POWER
81,UNIONBANK,163.00,-8.53,65.0,M-LC,9.82,166218.0,25378.0,19115.0,0.85,18.02,11.50,31.59,66.0,1.33,1.15,49.28,XY24,NTT,BANKS
12,BANKINDIA,190.00,-15.53,75.0,H-MC,15.22,216520.0,36712.0,75695.0,0.77,20.42,34.96,62.52,88.0,0.48,1.50,61.46,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.00,-15.53,75.0,H-MC,15.22,216520.0,36712.0,75695.0,0.77,20.42,34.96,62.52,88.0,0.48,1.50,61.46,XR,NTT,BANKS
80,UJJIVANSFB,60.00,129.87,73.0,H-SC,17.48,145368.0,2889.0,16630.0,-0.30,2.03,11.44,13.70,163.0,0.17,1.01,71.57,OX40N,NTT,BANKS
87,WHIRLPOOL,2270.00,-38.77,63.0,M-SC,3.19,126896.0,12098.0,81949.0,-0.86,10.54,64.58,81.92,223.0,0.15,0.88,50.22,XR,NTT,DURABLES
47,KPIGREEN,731.05,9.40,69.0,H-SC,5.28,130498.0,5201.0,53726.0,-0.83,4.15,41.17,47.03,141.0,0.10,0.90,63.40,MH,ATH,POWER
37,INDIAMART,4810.62,-53.15,58.0,H-SC,6.90,130915.0,7579.0,124042.0,0.61,6.15,94.75,106.72,119.0,0.06,0.91,30.96,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAMOTORS,1065.00,-53.17,19.0,X-LC,16.99,148848.0,-127675.0,237740.0,1.67,-46.17,159.72,39.81,54.0,-0.54,1.03,4.91,XY24,NTT,AUTO
13,BATAINDIA,2096.00,-37.87,35.0,X-SC,4.48,92877.0,-35793.0,81091.0,-4.05,-27.82,87.31,35.20,219.0,-0.44,0.64,6.60,X40,NTT,FOOTWEAR
24,DMART,5391.45,-17.48,35.0,X-LC,9.99,100958.0,-1628.0,28440.0,-1.21,-1.59,28.17,26.13,38.0,-0.06,0.70,23.58,X40N,ATH,FMCG
45,JSWINFRA,342.00,-20.69,36.0,X-MC,5.38,199125.0,-1222.0,31721.0,-0.34,-0.61,15.93,15.23,178.0,-0.04,1.38,31.44,X40N,NTT,REALTY
52,PAGEIND,51769.93,-27.78,38.0,X-MC,6.88,81640.0,-1870.0,21896.0,-0.40,-2.24,26.82,23.98,82.0,-0.09,0.56,3.45,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.50,-3.90,43.0,H-LC,1.00,154600.0,-31495.0,78970.0,-2.15,-16.92,51.08,25.51,15.0,-0.40,1.07,13.93,AR,ATH,ELECTRICAL
42,ITC,452.00,-37.08,60.0,X-LC,1.57,203644.0,3506.0,17391.0,-1.00,1.75,8.54,10.44,4.0,0.20,1.41,7.41,X40,NTT,FMCG
53,PGHH,17907.65,-30.41,39.0,X-MC,2.90,201570.0,750.0,67042.0,0.23,0.37,33.26,33.76,80.0,0.01,1.39,5.59,X40,ATH,FMCG
29,HAVELLS,2069.16,-6.89,42.0,X-MC,3.08,310842.0,-17149.0,123684.0,-0.83,-5.23,39.79,32.48,92.0,-0.14,2.15,5.60,X40,ATH,ELECTRICAL
86,VOLTAS,1530.00,1.86,55.0,X-MC,3.57,213570.0,21828.0,15932.0,-0.89,11.38,7.46,19.69,99.0,1.37,1.48,18.96,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
10,BAJAJHFL,181.50,-15.33,41.0,X-MC,5.88,178465.0,-22041.0,119197.0,-0.91,-10.99,66.79,48.45,90.0,-0.18,1.23,2.91,X40N,ATH,FINANCE
21,COLPAL,3767.14,-4.76,44.0,X-MC,8.07,222230.0,-41135.0,154494.0,0.26,-15.62,69.52,43.04,84.0,-0.27,1.54,3.17,XY25,ATH,FMCG
52,PAGEIND,51769.93,-27.78,38.0,X-MC,6.88,81640.0,-1870.0,21896.0,-0.40,-2.24,26.82,23.98,82.0,-0.09,0.56,3.45,X40,ATH,APPARELS
3,ACC,3906.00,-37.64,51.0,X-MC,2.70,185470.0,-52481.0,205130.0,-0.43,-22.06,110.60,64.15,174.0,-0.26,1.28,4.04,XY24,BTT,CEMENT
85,VBL,671.64,-19.74,46.0,X-LC,9.53,288982.0,-26860.0,139521.0,-1.39,-8.50,48.28,35.67,5.0,-0.19,2.00,4.17,X40N,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-44.90,52.0,X-SC,38.35,54509.0,-10497.0,161423.0,2.32,-16.15,296.14,232.18,198.0,-0.07,0.38,6.32,XY24,NTT,MISC
52,PAGEIND,51769.93,-27.78,38.0,X-MC,6.88,81640.0,-1870.0,21896.0,-0.40,-2.24,26.82,23.98,82.0,-0.09,0.56,3.45,X40,ATH,APPARELS
58,RELAXO,1176.00,-40.27,41.0,X-SC,4.69,81376.0,-63784.0,139714.0,-0.48,-43.94,171.69,52.31,136.0,-0.46,0.56,8.93,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-9.24,44.0,X-MC,2.98,89730.0,-948.0,15855.0,0.47,-1.05,17.67,16.44,101.0,-0.06,0.62,18.92,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-37.87,35.0,X-SC,4.48,92877.0,-35793.0,81091.0,-4.05,-27.82,87.31,35.20,219.0,-0.44,0.64,6.60,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4389.97,-28.04,53.0,X-LC,11.69,286136.0,-59820.0,126529.0,-1.33,-17.29,44.22,19.28,1.0,-0.47,1.98,5.78,X40,ATH,IT
40,INFY,2275.00,-17.80,59.0,X-LC,6.91,320458.0,7570.0,164107.0,0.14,2.42,51.21,54.87,3.0,0.05,2.22,10.90,X40,BTT,IT
42,ITC,452.00,-37.08,60.0,X-LC,1.57,203644.0,3506.0,17391.0,-1.00,1.75,8.54,10.44,4.0,0.20,1.41,7.41,X40,NTT,FMCG
85,VBL,671.64,-19.74,46.0,X-LC,9.53,288982.0,-26860.0,139521.0,-1.39,-8.50,48.28,35.67,5.0,-0.19,2.00,4.17,X40N,ATH,FMCG
1,ABB,7934.00,-40.84,45.0,H-LC,6.00,247128.0,-14491.0,133696.0,-1.71,-5.54,54.10,45.57,7.0,-0.11,1.71,5.16,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7155.56,64.0,L-SC,7.47,79604.0,-14206.0,90828.0,1.15,-15.14,114.10,81.67,269.0,-0.16,0.55,54.16,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,27.66,81331.0,-32218.0,72271.0,0.36,-28.37,88.86,35.27,268.0,-0.45,0.56,99.06,XR,NTT,HOTELS
50,MASFIN,398.61,-20.51,45.0,H-SC,9.13,90570.0,-7410.0,29010.0,0.17,-7.56,32.03,22.05,152.0,-0.26,0.63,31.65,XR,ATH,FINANCE
79,TTKPRESTIG,770.00,120.92,78.0,M-SC,1.64,95139.0,-5638.0,5727.0,12.26,-5.59,6.02,0.09,245.0,-0.98,0.66,24.99,OX40N,NTT,DURABLES
87,WHIRLPOOL,2270.00,-38.77,63.0,M-SC,3.19,126896.0,12098.0,81949.0,-0.86,10.54,64.58,81.92,223.0,0.15,0.88,50.22,XR,NTT,DURABLES


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAMMAANCAP,326.00,-197.73,79.0,M-SC,28.50,168597.0,19377.0,124795.0,-0.69,12.99,74.02,96.62,208.0,0.16,1.17,82.46,XY25,NTT,FINANCE
69,SONACOMS,806.63,-32.33,70.0,M-SC,8.31,84280.0,-16879.0,56881.0,-0.47,-16.69,67.49,39.54,202.0,-0.30,0.58,18.84,AR,ATH,AUTO
5,ANGELONE,3033.00,16.84,65.0,X-SC,5.34,206640.0,15634.0,42072.0,-2.21,8.19,20.36,30.21,157.0,0.37,1.43,30.78,X40N,NTT,FINANCE
12,BANKINDIA,190.00,-15.53,75.0,H-MC,15.22,216520.0,36712.0,75695.0,0.77,20.42,34.96,62.52,88.0,0.48,1.50,61.46,XR,NTT,BANKS
47,KPIGREEN,731.05,9.40,69.0,H-SC,5.28,130498.0,5201.0,53726.0,-0.83,4.15,41.17,47.03,141.0,0.10,0.90,63.40,MH,ATH,POWER


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.35
1,25,43.41
2,50,74.04


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.84
LC    31.82
MC    24.33
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.80
X40      16.10
XY25     11.96
XR       11.34
X40N     10.44
AR        7.71
OX40N     7.52
X200      1.74
X5K       1.41
SR        1.07
MH        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.27
X-LC    21.30
X-MC    18.57
M-SC    12.58
M-LC     5.00
X-SC     4.55
H-LC     4.52
H-MC     3.87
M-MC     1.57
L-SC     1.44
L-LC     1.00
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.26,-4.53,38.65
IT,12.38,-17.61,80.69
FINANCE,12.01,-8.89,58.24
MISC,6.83,-16.34,80.80
BANKS,6.68,-3.57,58.06
ELECTRICAL,5.70,-9.77,48.69
PAINTS,5.44,-17.28,34.76
INSURANCE,3.83,0.01,37.27
AUTO,2.74,-44.84,105.28


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3265365.0,22
XR,1312433.0,14
AR,1174567.0,9
X40,828024.0,12
X40N,650648.0,9
OX40N,559792.0,10
XY25,454972.0,8
SR,266680.0,2
X5K,113636.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3342155.0,25
M-SC,1225544.0,15
X-MC,1192227.0,14
X-LC,1060457.0,13
X-SC,522933.0,6
H-MC,321911.0,3
M-LC,286223.0,4
H-LC,285462.0,3
L-SC,253132.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1148528.0      6
           AR         837678.0      5
           XR         775745.0      7
M-SC       XY24       705245.0      6
X-MC       XY24       581291.0      4
X-LC       X40        457341.0      5
           XY24       306793.0      2
X-MC       X40        289592.0      6
X-SC       X40N       280419.0      4
H-SC       SR         266680.0      2
           OX40N      259798.0      4
X-LC       X40N       219311.0      3
H-LC       AR         212666.0      2
H-MC       XY24       177322.0      1
X-MC       XY25       170426.0      2
M-MC       XY24       165648.0      1
L-SC       XR         163099.0      2
X-SC       XY24       161423.0      1
X-MC       X40N       150918.0      2
M-SC       OX40N      141067.0      4
           XR         130214.0      2
           XY25       124795.0      1
           AR         124223.0      2
M-LC       X5K        113636.0      1
           XR         110633.0      1
L-SC       OX40N       90033.0      1
X-SC       X40         81091.0      1
X-LC       XY25        77012.0      3
H-MC       XR          75695.0      1
H-LC       X200        72796.0      1
H-MC       OX40N       68894.0      1
L-MC       XR          57047.0      1
H-SC       MH          53726.0      1
M-LC       XY25        42839.0      1
L-LC       XY25        39900.0      1
M-LC       XY24        19115.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
